# Pre-Processing: from raw data to model inputs

This notebook will showcase how the different pre-processing steps were performed. This document assumes that there is a folder named "Data" with a sub-folder "Raw" where the "sources" folder that was provided by ICE-CSIC exists.

In [1]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Data extraction

The data will be extracted using the functions under src.data.data_loader.py

In [2]:
from src.data.data_loader import extract_signals
from src.data.data_processor import load_data

# Define the base directory
base_directory = 'Data/Raw/sources'  # Change to where the tar files are located

# Extract the sources - uncomment the next line if you haven't done it yet
# extract_signals(base_directory)  # Generates a sources_extracted directory under 'Data/Raw'

# Store the data as a dictionary
data_dict = load_data('Data/Raw/sources_extracted')  # Change to where the extracted files are located

Extracting data: 100%|██████████| 1017/1017 [18:03<00:00,  1.07s/it]


## Interpolate data

Once the data has been correctly loaded, it must be interpolated to a common base so that all signals share the same channels.

In [3]:
from src.data.data_processor import interpolate_data, build_array

# Interpolate the data to a common base
interpolated_data_dict = interpolate_data(data_dict)

# Store the interpolated data as a numpy array for easier use
raw_data_spw0, mapping_0 = build_array(interpolated_data_dict, category="Intensity", spw=0)
raw_data_spw1, mapping_1 = build_array(interpolated_data_dict, category="Intensity", spw=1)

if np.all(np.array(mapping_0)[:, :-1] == np.array(mapping_1)[:, :-1]):
    mapping = np.array(mapping_0)[:, :-1]
    raw_data = np.concatenate((raw_data_spw0, raw_data_spw1), axis=1)
else:
    raise ValueError("Mappings for intensity's spw0 and spw1 do not match.")

residual_spw0, mapping_0 = build_array(interpolated_data_dict, category="Residual", spw=0)
residual_spw1, mapping_1 = build_array(interpolated_data_dict, category="Residual", spw=1)

if np.all(np.array(mapping_0)[:, :-1] == np.array(mapping_1)[:, :-1]):
    mapping = np.array(mapping_0)[:, :-1]
    residual = np.concatenate((residual_spw0, residual_spw1), axis=1)
else:
    raise ValueError("Mappings for residual's spw0 and spw1 do not match.")

Interpolating data: 100%|██████████| 641/641 [00:08<00:00, 78.18it/s] 


## Data filtering

Three different filters are used in this work:
- Threshold filter
- Subtraction filter
- Savitzky-Golay filter

In [4]:
from src.data.data_processor import filter_data

# Apply threhsold-filtering
threshold_filtered_data = filter_data(raw_data, filter_type="sigma", residual=residual)

# Apply subtraction-filtering
subtraction_filtered_data = filter_data(raw_data, filter_type="subtraction", residual=residual)

# Apply Savitzky-Golay filtering
savgol_params = {'window_length':25, 'polyorder':3}
savitzky_golay_filtered_data = filter_data(raw_data, filter_type="savgol", **savgol_params)

## Red-shift correction

The RedShiftCorrector class will identify a most-likely velocity for all sources when provided with the following data:
- raw_data (array)
- mapping (list)
- residual (array)

It will by default compare the velocities found for the raw-data, threshold-filtered data and subtraction-filtered data with a margin of +/- 5 km/s.

The output variable will be a pandas DataFrame with the following columns:
- Source
- Core
- None_velocity (velocity for the raw data)
- threshold_velocity (velocity for the threshold-filtered data)
- subtraction_velocity (velocity for the subtraction-filtered data)

In [5]:
from src.data.data_processor import RedShiftCorrector
from src.features.transformations import shift_signal

# Retrieve the common frequency base
region = list(interpolated_data_dict.keys())[0]
core = list(interpolated_data_dict[region].keys())[0]

frequency_spw0 = interpolated_data_dict[region][core][0]['Frequency']
frequency_spw1 = interpolated_data_dict[region][core][1]['Frequency']
frequency = np.concatenate((frequency_spw0, frequency_spw1), axis=0)

# Apply red-shift correction to the data
velocity_results = RedShiftCorrector(frequency=frequency).fit(raw_data, mapping, residual)

# Retrieve the indices for which a velocity was not found
no_velocity_indices = velocity_results[velocity_results.isna().any(axis=1)].index.to_list()

# Shift the data
shifted_signals = np.array([shift_signal(frequency, raw_data[i], velocity_results.loc[i].values) for i in range(raw_data.shape[0]) if i not in no_velocity_indices])
shifted_residual = np.array([shift_signal(frequency, residual[i], velocity_results.loc[i].values) for i in range(raw_data.shape[0]) if i not in no_velocity_indices])
threshold_shifted_signals = np.array([shift_signal(frequency, threshold_filtered_data[i], velocity_results.loc[i].values) for i in range(raw_data.shape[0]) if i not in no_velocity_indices])
subtraction_shifted_signals = np.array([shift_signal(frequency, subtraction_filtered_data[i], velocity_results.loc[i].values) for i in range(raw_data.shape[0]) if i not in no_velocity_indices])

Filtering method: None


Shifting signals: 100%|██████████| 4396/4396 [24:09<00:00,  3.03it/s]


Filtering method: subtraction


Shifting signals: 100%|██████████| 4396/4396 [24:18<00:00,  3.01it/s]


Filtering method: threshold


c:\Users\javia\OneDrive\Desktop\Programing\MSc-Thesis\src\data\data_processor.py:665: SyntaxWarning: invalid escape sequence '\D'
  main_ax.set_ylabel(r'$\Delta$V [km/s]')


ValueError: threshold is not recognised as an established filter type

The data should be saved for it to be used in other notebooks. Feel free to not run the next cell if you do not need it.

In [ ]:
# Store the data
processed_data = {
    'raw_data': raw_data,
    'residual': residual,
    'threshold_filtered_data': threshold_filtered_data,
    'subtraction_filtered_data': subtraction_filtered_data,
    'savitzky_golay_filtered_data': savitzky_golay_filtered_data,
    'shifted_signals': shifted_signals,
    'shifted_residual': shifted_residual,
    'threshold_shifted_signals': threshold_shifted_signals,
    'subtraction_shifted_signals': subtraction_shifted_signals,
    'velocity_results': velocity_results,
    'no_velocity_indices': no_velocity_indices,
    'mapping': mapping,
    'frequency': frequency
}

with open('Data/Processed/processed_data.pkl', 'wb') as f:
    pickle.dump(processed_data, f)

## Dimensionality reduction

Two methods of dimensionality reduction were used in this study:
- PCA
- UMAP

### PCA

In [ ]:
from sklearn.decomposition import PCA

for data in [shifted_signals, threshold_shifted_signals, subtraction_shifted_signals]:
    # Reduce the data
    pca = PCA(n_components=100).fit(data)
    
    # Plot explained variance ratio
    plt.plot(range(1, 101), np.cumsum(pca.explained_variance_ratio_), marker='o')

plt.legend(["Raw data", "Threshold-filtered data", "Subtraction-filtered data"])
plt.xlabel("Number of components")
plt.ylabel("Cumulative explained variance ratio")
plt.show()

In [ ]:
# Raw data
pca_data = PCA(n_components=30).fit_transform(shifted_signals)
np.save("Data/Processed/raw_pca_data.npy", pca_data)

# Threshold-filtered data
pca_threshold_data = PCA(n_components=30).fit_transform(threshold_shifted_signals)
np.save("Data/Processed/threshold_pca_data.npy", pca_threshold_data)

# Subtraction-filtered data
pca_subtraction_data = PCA(n_components=30).fit_transform(subtraction_shifted_signals)
np.save("Data/Processed/subtraction_pca_data.npy", pca_subtraction_data)

### UMAP

In [ ]:
from umap import UMAP

for data in [shifted_signals, threshold_shifted_signals, subtraction_shifted_signals]:
    # Reduce the data
    umap = UMAP(n_components=2).fit(data)
    
    # Plot explained variance ratio
    plt.scatter(umap.embedding_[:,0], umap.embedding_[:,1], s=1, alpha=0.5)

In [ ]:
# Raw data
umap_data = UMAP(n_components=2).fit_transform(shifted_signals)
np.save("Data/Processed/raw_umap_data.npy", umap_data)

# Threshold-filtered data
umap_threshold_data = UMAP(n_components=2).fit_transform(threshold_shifted_signals)
np.save("Data/Processed/threshold_umap_data.npy", umap_threshold_data)

# Subtraction-filtered data
umap_subtraction_data = UMAP(n_components=2).fit_transform(subtraction_shifted_signals)
np.save("Data/Processed/subtraction_umap_data.npy", umap_subtraction_data)